In [1]:
import tensorflow as tf
import pandas as pd
import os
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [2]:
data_dir = "data/numbers"
image_size = (45, 45)
color_mode = "grayscale"

X = []
y = []
class_names = sorted(os.listdir(data_dir))  # get class folders
class_to_label = {name: idx for idx, name in enumerate(class_names)}

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    for fname in os.listdir(class_dir):
        if fname.lower().endswith(".jpg"):
            img_path = os.path.join(class_dir, fname)
            img = load_img(img_path, target_size=image_size, color_mode=color_mode)
            img_array = img_to_array(img) / 255.0
            X.append(img_array)
            y.append(class_to_label[class_name])

X = np.array(X)
y = np.array(y)

#Save
np.savez_compressed("new_data_nums.npz", X=X, y=y)

#load
#data = np.load("new_data_nums.npz")
#X_loaded, y_loaded = data["X"], data["y"]

print("X shape:", X.shape)  # 4D array med num_images, height, width, channels)
print("y shape:", y.shape)  #1D array där nummer klass tilldelas till varje bild, tex X[1]=5 = bilden med index 1 är en 5a.


X shape: (94257, 45, 45, 1)
y shape: (94257,)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

X_train = X_train.reshape(len(X_train), -1)
X_val = X_val.reshape(len(X_val), -1)
X_test = X_test.reshape(len(X_test), -1)

#Flattena datan för att få rätt antal dimmensioner till modellerna
print("Flattened Train shape:", X_train.shape, y_train.shape)
print("Flattened Val shape:", X_val.shape, y_val.shape)
print("Flattened Test shape:", X_test.shape, y_test.shape)


Train shape: (64094, 45, 45, 1) (64094,)
Val shape: (11311, 45, 45, 1) (11311,)
Test shape: (18852, 45, 45, 1) (18852,)
Flattened Train shape: (64094, 2025) (64094,)
Flattened Val shape: (11311, 2025) (11311,)
Flattened Test shape: (18852, 2025) (18852,)


In [4]:
#Scaling
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_val)

In [5]:
#params

#param_log = {
#    'C': [0.1, 1, 10],
#    'solver': ['lbfgs', 'saga'],
#    'max_iter': [500, 1000]
#}

param_rf = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5]
}
    
param_et = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5]
}

#träna modell
#grid_log = GridSearchCV(LogisticRegression(), param_log, cv=3, scoring='accuracy', n_jobs=-1)
grid_rf  = GridSearchCV(RandomForestClassifier(), param_rf, cv=3, scoring='accuracy', n_jobs=-1)
grid_extra = GridSearchCV(ExtraTreesClassifier(), param_et, cv=3, scoring='accuracy', n_jobs=-1)

In [6]:
#grid_log.fit(X_train, y_train)
grid_rf.fit(X_train, y_train)
grid_extra.fit(X_train, y_train)

#best params
print("Best RandomForest params:", grid_rf.best_params_)
print("Best ExtraTrees params:", grid_extra.best_params_)

Best RandomForest params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best ExtraTrees params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}


In [7]:
#log_pred   = grid_log.predict(X_val)
ran_pred   = grid_rf.predict(X_val)
extra_pred = grid_extra.predict(X_val)

In [9]:
print("RandomForest RMSE: ", root_mean_squared_error(y_val, ran_pred))
print("ExtraTrees RMSE: ", root_mean_squared_error(y_val, extra_pred))

RandomForest RMSE:  0.5975667514646512
ExtraTrees RMSE:  0.41328825152966125


In [10]:
# Utvärdering
#print("=== Logistic Regression ===")
#print(classification_report(y_val, log_pred))

print("=== Random Forest ===")
print(classification_report(y_val, ran_pred))

print("=== Extra Trees ===")
print(classification_report(y_val, extra_pred))


=== Random Forest ===
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       806
           1       1.00      0.84      0.91      3195
           2       0.83      1.00      0.91      3135
           3       0.95      0.99      0.97      1258
           4       1.00      0.95      0.97       885
           5       0.99      0.94      0.96       418
           6       0.99      0.93      0.96       397
           7       1.00      0.94      0.97       357
           8       1.00      0.94      0.97       381
           9       1.00      0.94      0.97       479

    accuracy                           0.94     11311
   macro avg       0.98      0.94      0.96     11311
weighted avg       0.95      0.94      0.94     11311

=== Extra Trees ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       806
           1       1.00      1.00      1.00      3195
           2       0.98      1.00   